# Create Agent with API Schema and User Confirmation

In this notebook, we will demonstrate the creation of an agent designed to assist with insurance claims processing. The agent is built to support insurance employees by efficiently handling tasks such as reviewing open claims, retrieving details for specific claims, identifying outstanding claim documents, and sending reminders to policyholders. The agent is capable of managing these tasks individually or as part of a multi-step process, ensuring streamlined operations. Specifically, the agent can:

- Retrieve Open Claims
- Access Claim Details
- Identify Claim Outstanding Documents
- Send Claim Reminders (with User Confirmation Enable)

Additionally, we will leverage Amazon Bedrock latest capabilities to define functions and incorporate user confirmation before executing grouped actions, further enhancing the agent's reliability and user control.

![](../images/architecture_11_3.png)

## Pre-requisites
Before starting, let's import the required packages and configure the support variables

In [1]:
import logging
import boto3
import time
import zipfile
from io import BytesIO
import json
import uuid
import pprint

In [2]:
# setting logger
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

In [3]:
# get boto3 clients for required AWS services
sts_client = boto3.client('sts')
iam_client = boto3.client('iam')
s3_client = boto3.client('s3')
lambda_client = boto3.client('lambda')
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')

In [4]:
session = boto3.session.Session()
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]
region, account_id

('us-west-2', '114538913250')

In [17]:
# Generate random prefix for unique IAM roles, agent name and S3 Bucket and 
# assign variables
suffix = f"{region}-{account_id}4"
agent_name = "ins-claims-agent-user-conf"
agent_alias_name = "workshop-alias"
bucket_name = f'{agent_name}-{suffix}'
bucket_key = f'{agent_name}-schema.json'
schema_name = 'insurance_claims_agent_openapi_schema.json'
schema_arn = f'arn:aws:s3:::{bucket_name}/{bucket_key}'
bedrock_agent_bedrock_allow_policy_name = f"{agent_name}-allow-{suffix}"
bedrock_agent_s3_allow_policy_name = f"{agent_name}-s3-allow-{suffix}"
lambda_role_name = f'{agent_name}-lambda-role-{suffix}'
agent_role_name = f'AmazonBedrockExecutionRoleForAgents_{suffix}'
lambda_code_path = "lambda_function.py"
lambda_name = f'{agent_name}-{suffix}'
model_id = "anthropic.claude-3-sonnet-20240229-v1:0"

## Create S3 bucket and upload API Schema with the Required Confirmation field enabled

Before uploading our Schema file in the OpenAPI standard, it is important to understand the configuration that enables user confirmation prior to API execution. Notice that in the `/notify` path, after the "responses" section, we have the field `"x-requireConfirmation":"ENABLED"`. This is an optional field, and the default setting is `DISABLED`. Enable this field to request confirmation from the user before the action is invoked.

For this example, we will enable only the `/notify` path to request user confirmation before it is executed.

Now, agents require an API Schema stored on s3. Let's create an S3 bucket to store the file and upload the file to the newly created bucket:

In [18]:
# Create S3 bucket for Open API schema
if region == "us-east-1":
    s3bucket = s3_client.create_bucket(
        Bucket=bucket_name
    )
else:
    s3bucket = s3_client.create_bucket(
        Bucket=bucket_name,
        CreateBucketConfiguration={ 'LocationConstraint': region } 
    )

In [19]:
# Upload Open API schema to this s3 bucket
s3_client.upload_file(schema_name, bucket_name, bucket_key)

## Create Lambda function for Action Group
Let's now create the lambda function required by the agent action group. We first need to create the lambda IAM role and it's policy. After that, we package the lambda function into a ZIP format to create the function

In [20]:
# Create IAM Role for the Lambda function
try:
    assume_role_policy_document = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Action": "bedrock:InvokeModel",
                "Principal": {
                    "Service": "lambda.amazonaws.com"
                },
                "Action": "sts:AssumeRole"
            }
        ]
    }

    assume_role_policy_document_json = json.dumps(assume_role_policy_document)

    lambda_iam_role = iam_client.create_role(
        RoleName=lambda_role_name,
        AssumeRolePolicyDocument=assume_role_policy_document_json
    )

    # Pause to make sure role is created
    time.sleep(10)
except:
    lambda_iam_role = iam_client.get_role(RoleName=lambda_role_name)

iam_client.attach_role_policy(
    RoleName=lambda_role_name,
    PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'
)

{'ResponseMetadata': {'RequestId': '5c3589e0-9230-4162-b9e1-e45f41afd77c',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 22 Nov 2024 16:41:54 GMT',
   'x-amzn-requestid': '5c3589e0-9230-4162-b9e1-e45f41afd77c',
   'content-type': 'text/xml',
   'content-length': '212'},
  'RetryAttempts': 0}}

Take a look at the Lambda function code that will be used as an Action group for the agent

In [21]:
!pygmentize lambda_function.py

#!/usr/bin/env python3
# Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
# SPDX-License-Identifier: MIT-0
import json


def get_named_parameter(event, name):
    return next(item for item in event['parameters'] if item['name'] == name)['value']


def get_named_property(event, name):
    return next(
        item for item in
        event['requestBody']['content']['application/json']['properties']
        if item['name'] == name)['value']


def claim_detail(payload):
    claim_id = payload['parameters'][0]['value']
    if claim_id == 'claim-857':
        return {
            "response": {
                "claimId": claim_id,
                "createdDate": "21-Jul-2023",
                "lastActivityDate": "25-Jul-2023",
                "status": "Open",
                "policyType": "Vehicle"
            }
        }
    elif claim_id == 'claim-006':
        return {
            "response": {
                "claimId": claim_id,
                "createdDate": "20-May-2

In [22]:
# Package up the lambda function code
s = BytesIO()
z = zipfile.ZipFile(s, 'w')
z.write(lambda_code_path)
z.close()
zip_content = s.getvalue()

# Create Lambda Function
lambda_function = lambda_client.create_function(
    FunctionName=lambda_name,
    Runtime='python3.12',
    Timeout=180,
    Role=lambda_iam_role['Role']['Arn'],
    Code={'ZipFile': zip_content},
    Handler='lambda_function.lambda_handler'
)

## Create Agent
We will now create the agent. To do so, we first need to create the agent policies that allow bedrock model invocation and the agent IAM role with the policy associated to it. We will allow this agent to invoke the Claude Sonnet model.

In [23]:
# Create IAM policies for agent

bedrock_agent_bedrock_allow_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AmazonBedrockAgentBedrockFoundationModelPolicy",
            "Effect": "Allow",
            "Action": "bedrock:InvokeModel",
            "Resource": [
                f"arn:aws:bedrock:{region}::foundation-model/{model_id}"
            ]
        }
    ]
}

bedrock_policy_json = json.dumps(bedrock_agent_bedrock_allow_policy_statement)

agent_bedrock_policy = iam_client.create_policy(
    PolicyName=bedrock_agent_bedrock_allow_policy_name,
    PolicyDocument=bedrock_policy_json
)



Next, we will create a policy document that allows fetching of the Agent's OpenAPI schema from S3:

In [24]:
bedrock_agent_s3_allow_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AllowAgentAccessOpenAPISchema",
            "Effect": "Allow",
            "Action": ["s3:GetObject"],
            "Resource": [
                schema_arn
            ]
        }
    ]
}


bedrock_agent_s3_json = json.dumps(bedrock_agent_s3_allow_policy_statement)
agent_s3_schema_policy = iam_client.create_policy(
    PolicyName=bedrock_agent_s3_allow_policy_name,
    Description=f"Policy to allow invoke Lambda that was provisioned for it.",
    PolicyDocument=bedrock_agent_s3_json
)

Finally, create a role with the above two policies attached

In [25]:
# Create IAM Role for the agent and attach IAM policies
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [{
          "Effect": "Allow",
          "Principal": {
            "Service": "bedrock.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
    }]
}

assume_role_policy_document_json = json.dumps(assume_role_policy_document)
agent_role = iam_client.create_role(
    RoleName=agent_role_name,
    AssumeRolePolicyDocument=assume_role_policy_document_json
)

# Pause to make sure role is created
time.sleep(10)
    
iam_client.attach_role_policy(
    RoleName=agent_role_name,
    PolicyArn=agent_bedrock_policy['Policy']['Arn']
)

iam_client.attach_role_policy(
    RoleName=agent_role_name,
    PolicyArn=agent_s3_schema_policy['Policy']['Arn']
)

{'ResponseMetadata': {'RequestId': '06c218da-853d-4e3a-ba58-2aa381bbf1b5',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 22 Nov 2024 16:42:06 GMT',
   'x-amzn-requestid': '06c218da-853d-4e3a-ba58-2aa381bbf1b5',
   'content-type': 'text/xml',
   'content-length': '212'},
  'RetryAttempts': 0}}

## Creating Agent
Once the needed IAM role is created, we can use the bedrock agent client to create a new agent. To do so we use the `create_agent` function. It requires an agent name, underlying foundation model and instructions. You can also provide an agent description. Note that the agent created is not yet prepared. We will focus on preparing the agent and then using it to invoke actions and use other APIs

In [26]:
# Create Agent
agent_instruction = """
You are an agent that can handle various tasks related to insurance claims, including looking up claim 
details, finding what paperwork is outstanding, and sending reminders. Only send reminders if you have been 
explicitly requested to do so. If an user asks about your functionality, provide guidance in natural language 
and do not include function names on the output."""

response = bedrock_agent_client.create_agent(
    agentName=agent_name,
    agentResourceRoleArn=agent_role['Role']['Arn'],
    description="Agent for handling insurance claims.",
    idleSessionTTLInSeconds=1800,
    foundationModel=model_id,
    instruction=agent_instruction,
)

Looking at the created agent, we can see its status and agent id

In [27]:
response

{'ResponseMetadata': {'RequestId': '86a801c3-c73a-40db-bde3-2ce54c70adbd',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Fri, 22 Nov 2024 16:42:07 GMT',
   'content-type': 'application/json',
   'content-length': '943',
   'connection': 'keep-alive',
   'x-amzn-requestid': '86a801c3-c73a-40db-bde3-2ce54c70adbd',
   'x-amz-apigw-id': 'BqEe7GJIvHcEvGw=',
   'x-amzn-trace-id': 'Root=1-6740b45f-0dad05295dadf95d6e054478'},
  'RetryAttempts': 0},
 'agent': {'agentArn': 'arn:aws:bedrock:us-west-2:114538913250:agent/V5AGCELWHN',
  'agentId': 'V5AGCELWHN',
  'agentName': 'ins-claims-agent-user-conf',
  'agentResourceRoleArn': 'arn:aws:iam::114538913250:role/AmazonBedrockExecutionRoleForAgents_us-west-2-1145389132504',
  'agentStatus': 'CREATING',
  'createdAt': datetime.datetime(2024, 11, 22, 16, 42, 7, 284971, tzinfo=tzlocal()),
  'description': 'Agent for handling insurance claims.',
  'foundationModel': 'anthropic.claude-3-sonnet-20240229-v1:0',
  'idleSessionTTLInSeconds': 1800,
  'in

Let's now store the agent id in a local variable to use it on the next steps

In [28]:
agent_id = response['agent']['agentId']
agent_id

'V5AGCELWHN'

## Create Agent Action Group
We will now create and agent action group that uses the lambda function and API schema files created before.
The `create_agent_action_group` function provides this functionality. We will use `DRAFT` as the agent version since we haven't yet create an agent version or alias. To inform the agent about the action group functionalities, we will provide an action group description containing the functionalities of the action group.

In [29]:
# Pause to make sure agent is created
time.sleep(30)
# Now, we can configure and create an action group here:
agent_action_group_response = bedrock_agent_client.create_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupExecutor={
        'lambda': lambda_function['FunctionArn']
    },
    actionGroupName='ClaimManagementActionGroup',
    apiSchema={
        's3': {
            's3BucketName': bucket_name,
            's3ObjectKey': bucket_key
        }
    },
    description='Actions for listing claims, identifying missing paperwork, sending reminders'
)

In [30]:
agent_action_group_response

{'ResponseMetadata': {'RequestId': 'a6cbc81d-4b8c-4078-a9c2-e30fa8498bcf',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 22 Nov 2024 16:42:45 GMT',
   'content-type': 'application/json',
   'content-length': '633',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'a6cbc81d-4b8c-4078-a9c2-e30fa8498bcf',
   'x-amz-apigw-id': 'BqEkxE61PHcElEw=',
   'x-amzn-trace-id': 'Root=1-6740b484-00014f77390cd5d03bd569b4'},
  'RetryAttempts': 0},
 'agentActionGroup': {'actionGroupExecutor': {'lambda': 'arn:aws:lambda:us-west-2:114538913250:function:ins-claims-agent-user-conf-us-west-2-1145389132504'},
  'actionGroupId': 'DO9GOVCQRD',
  'actionGroupName': 'ClaimManagementActionGroup',
  'actionGroupState': 'ENABLED',
  'agentId': 'V5AGCELWHN',
  'agentVersion': 'DRAFT',
  'apiSchema': {'s3': {'s3BucketName': 'ins-claims-agent-user-conf-us-west-2-1145389132504',
    's3ObjectKey': 'ins-claims-agent-user-conf-schema.json'}},
  'createdAt': datetime.datetime(2024, 11, 22, 16, 42, 45, 37708

## Allowing Agent to invoke Action Group Lambda
Before using our action group, we need to allow our agent to invoke the lambda function associated to the action group. This is done via resource-based policy. Let's add the resource-based policy to the lambda function created

In [31]:
# Create allow invoke permission on lambda
response = lambda_client.add_permission(
    FunctionName=lambda_name,
    StatementId='allow_bedrock',
    Action='lambda:InvokeFunction',
    Principal='bedrock.amazonaws.com',
    SourceArn=f"arn:aws:bedrock:{region}:{account_id}:agent/{agent_id}",
)

## Preparing Agent
Let's create a DRAFT version of the agent that can be used for internal testing.

In [32]:
agent_prepare = bedrock_agent_client.prepare_agent(agentId=agent_id)
agent_prepare

{'ResponseMetadata': {'RequestId': '92b54b57-ddf1-45a7-901d-cbe3a504801d',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Fri, 22 Nov 2024 16:42:45 GMT',
   'content-type': 'application/json',
   'content-length': '119',
   'connection': 'keep-alive',
   'x-amzn-requestid': '92b54b57-ddf1-45a7-901d-cbe3a504801d',
   'x-amz-apigw-id': 'BqEk7GwjPHcEpQQ=',
   'x-amzn-trace-id': 'Root=1-6740b485-1ac02870399db5f2780892a3'},
  'RetryAttempts': 0},
 'agentId': 'V5AGCELWHN',
 'agentStatus': 'PREPARING',
 'agentVersion': 'DRAFT',
 'preparedAt': datetime.datetime(2024, 11, 22, 16, 42, 45, 732374, tzinfo=tzlocal())}

## Create Agent alias
We will now create an alias of the agent that can be used to deploy the agent.

In [33]:
# Pause to make sure agent is prepared
time.sleep(30)
agent_alias = bedrock_agent_client.create_agent_alias(
    agentId=agent_id,
    agentAliasName=agent_alias_name
)

In [34]:
agent_alias

{'ResponseMetadata': {'RequestId': '71ef4159-4456-456b-98d4-a1dea09df10a',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Fri, 22 Nov 2024 16:43:15 GMT',
   'content-type': 'application/json',
   'content-length': '340',
   'connection': 'keep-alive',
   'x-amzn-requestid': '71ef4159-4456-456b-98d4-a1dea09df10a',
   'x-amz-apigw-id': 'BqEppGAJvHcEZrw=',
   'x-amzn-trace-id': 'Root=1-6740b4a3-11aea10d76fa6a3d56e25a4f'},
  'RetryAttempts': 0},
 'agentAlias': {'agentAliasArn': 'arn:aws:bedrock:us-west-2:114538913250:agent-alias/V5AGCELWHN/SFOGUF2JF4',
  'agentAliasId': 'SFOGUF2JF4',
  'agentAliasName': 'workshop-alias',
  'agentAliasStatus': 'CREATING',
  'agentId': 'V5AGCELWHN',
  'createdAt': datetime.datetime(2024, 11, 22, 16, 43, 15, 889336, tzinfo=tzlocal()),
  'routingConfiguration': [{}],
  'updatedAt': datetime.datetime(2024, 11, 22, 16, 43, 15, 889336, tzinfo=tzlocal())}}

## Create a simple function to Invoke the Agent

In [35]:
def simple_agent_invoke(input_text, agent_id, agent_alias_id, session_id=None, enable_trace=False, end_session=False):
    
    agentResponse = bedrock_agent_runtime_client.invoke_agent(
        inputText=input_text,
        agentId=agent_id,
        agentAliasId=agent_alias_id, 
        sessionId=session_id,
        enableTrace=enable_trace, 
        endSession= end_session
    )
    
    event_stream = agentResponse['completion']
    try:
        for event in event_stream:  
            if 'returnControl' in event:
                pprint.pp(event)
                
                return event
            elif 'chunk' in event:
                data = event['chunk']['bytes']
                logger.info(f"Final answer ->\n{data.decode('utf8')}")
                agent_answer = data.decode('utf8')
                end_event_received = True
                
                print(agent_answer)
                # End event indicates that the request finished successfully
                return event
            elif 'trace' in event:
                logger.info(json.dumps(event['trace'], indent=2))
            else:
                raise Exception("unexpected event.", event)
    except Exception as e:
        raise Exception("unexpected event.", e)

## Extract the agentAliasId from the response

In [36]:
time.sleep(30)

agent_alias_id = agent_alias['agentAlias']['agentAliasId']

## Invoke the Agent for an API that does not have User Confirmation Enabled

As the first test, we will invoke our agent with a prompt that we know is for an API where user confirmation has not been enabled. Now that we've created the agent, let's use the `bedrock-agent-runtime` client to invoke this agent and perform some tasks.

In [37]:
## create a random id for session initiator id
session_id:str = str(uuid.uuid1())
enable_trace:bool = False
end_session:bool = False

simple_agent_invoke("what are the open claims?", agent_id, agent_alias_id, session_id)

[2024-11-22 16:43:59,496] p5975 {1476644290.py:21} INFO - Final answer ->
The open insurance claims are:

- Claim ID: claim-006, Policy Holder ID: A945684
- Claim ID: claim-857, Policy Holder ID: A645987  
- Claim ID: claim-334, Policy Holder ID: A987654


The open insurance claims are:

- Claim ID: claim-006, Policy Holder ID: A945684
- Claim ID: claim-857, Policy Holder ID: A645987  
- Claim ID: claim-334, Policy Holder ID: A987654


{'chunk': {'bytes': b'The open insurance claims are:\n\n- Claim ID: claim-006, Policy Holder ID: A945684\n- Claim ID: claim-857, Policy Holder ID: A645987  \n- Claim ID: claim-334, Policy Holder ID: A987654'}}

## Invoke the Agent for an API that has User Confirmation Enabled

In [38]:
## create a random id for session initiator id
session_id:str = str(uuid.uuid1())
enable_trace:bool = False
end_session:bool = False

event = simple_agent_invoke("Please send a reminder to the insured about claim claim-006 only. Please reply indicating whether the reminder was sent or not", agent_id, agent_alias_id, session_id)

{'returnControl': {'invocationId': '3b8f0f5b-6656-4a23-b659-1c8ec1524672-uc',
                   'invocationInputs': [{'apiInvocationInput': {'actionGroup': 'ClaimManagementActionGroup',
                                                                'actionInvocationType': 'USER_CONFIRMATION',
                                                                'apiPath': '/notify',
                                                                'httpMethod': 'POST',
                                                                'parameters': [],
                                                                'requestBody': {'content': {'application/json': {'properties': [{'name': 'claimId',
                                                                                                                                 'type': 'string',
                                                                                                                                 'value': 'claim-006'},
   

## Invoking the Agent with the function results and Allowed execution

Finally, we need to invoke the agent passing the function results as a parameter. This lets us use the agent for generating the final response. Note that if user confirmation is enabled for the function or API, the user will be presented with the option to `CONFIRM` or `DENY` the action specified in the response, within the `confirmationState` field. For this example, we will use the `confirmationState` as CONFIRM, which means that the user has confirmed the action, allowing it to be executed.

In [39]:
raw_response_with_allowed = bedrock_agent_runtime_client.invoke_agent(
    agentId=agent_id,
    agentAliasId=agent_alias_id, 
    sessionId=session_id,
    enableTrace=enable_trace, 
    sessionState={
        'invocationId': event["returnControl"]["invocationId"],
        'returnControlInvocationResults': [{
                'apiResult': {
                    'actionGroup': event["returnControl"]["invocationInputs"][0]["apiInvocationInput"]["actionGroup"],
                    'apiPath': event["returnControl"]["invocationInputs"][0]["apiInvocationInput"]["apiPath"],
                    'confirmationState': 'CONFIRM',
                    'httpMethod': event["returnControl"]["invocationInputs"][0]["apiInvocationInput"]["httpMethod"],
                    'httpStatusCode': 200,
                    'responseBody': {
                        "TEXT": {
                            'body': ''
                        }
                    }
                }
        }]}
)

print(raw_response_with_allowed)

{'ResponseMetadata': {'RequestId': '8b897ce8-082d-4337-85c4-ddab19667504', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Fri, 22 Nov 2024 16:44:04 GMT', 'content-type': 'application/vnd.amazon.eventstream', 'transfer-encoding': 'chunked', 'connection': 'keep-alive', 'x-amzn-requestid': '8b897ce8-082d-4337-85c4-ddab19667504', 'x-amz-bedrock-agent-session-id': 'f9023058-a8f0-11ef-ab73-52a1e22075a3', 'x-amzn-bedrock-agent-content-type': 'application/json'}, 'RetryAttempts': 0}, 'contentType': 'application/json', 'sessionId': 'f9023058-a8f0-11ef-ab73-52a1e22075a3', 'completion': <botocore.eventstream.EventStream object at 0x7fa748c50510>}


In [40]:
%%time
event_stream = raw_response_with_allowed['completion']
for event_allowed in event_stream:
    print(event_allowed)

{'chunk': {'bytes': b'The reminder for claim claim-006 to submit the pending document AccidentImages was sent successfully. The tracking ID for the reminder is 50e8400-e29b-41d4-a716-446655440000.'}}
CPU times: user 2.11 ms, sys: 300 μs, total: 2.41 ms
Wall time: 1.72 s


## Invoking an Agent with function results and execution is NOT allowed

As shown in the previous example, we are going to call the same method `invoke_agent`, but this time we will pass the option as `DENY`, which means the user did not permit the execution.

In [41]:
raw_response_with_not_allowed = bedrock_agent_runtime_client.invoke_agent(
    agentId=agent_id,
    agentAliasId=agent_alias_id, 
    sessionId=session_id,
    enableTrace=enable_trace, 
    sessionState={
        'invocationId': event["returnControl"]["invocationId"],
        'returnControlInvocationResults': [{
                'apiResult': {
                    'actionGroup': event["returnControl"]["invocationInputs"][0]["apiInvocationInput"]["actionGroup"],
                    'apiPath': event["returnControl"]["invocationInputs"][0]["apiInvocationInput"]["apiPath"],
                    'confirmationState': 'DENY',
                    'httpMethod': event["returnControl"]["invocationInputs"][0]["apiInvocationInput"]["httpMethod"],
                    'httpStatusCode': 200,
                    'responseBody': {
                        "TEXT": {
                            'body': ''
                        }
                    }
                }
        }]}
)

print(raw_response_with_not_allowed)

{'ResponseMetadata': {'RequestId': '763f9ff8-983d-49fc-a248-d655b2efd0e2', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Fri, 22 Nov 2024 16:44:06 GMT', 'content-type': 'application/vnd.amazon.eventstream', 'transfer-encoding': 'chunked', 'connection': 'keep-alive', 'x-amzn-requestid': '763f9ff8-983d-49fc-a248-d655b2efd0e2', 'x-amz-bedrock-agent-session-id': 'f9023058-a8f0-11ef-ab73-52a1e22075a3', 'x-amzn-bedrock-agent-content-type': 'application/json'}, 'RetryAttempts': 0}, 'contentType': 'application/json', 'sessionId': 'f9023058-a8f0-11ef-ab73-52a1e22075a3', 'completion': <botocore.eventstream.EventStream object at 0x7fa765d89a50>}


In [43]:
%%time
event_stream = raw_response_with_not_allowed['completion']
for event_not_allowed in event_stream:
    print(event_not_allowed)

CPU times: user 289 μs, sys: 40 μs, total: 329 μs
Wall time: 266 μs


After sending the `DENY` option in the `confirmationState`, you should receive the following message:

`Unfortunately, I was unable to send the reminder for claim claim-006 due to an access issue.`

### Clean up (optional)
The next steps are optional and demonstrate how to delete our agent. To delete the agent we need to:
1. update the action group to disable it
2. delete agent action group
3. delete agent alias
4. delete agent
5. delete lambda function
6. empty created s3 bucket
7. delete s3 bucket

In [ ]:
 # This is not needed, you can delete agent successfully after deleting alias only
# Additionaly, you need to disable it first

action_group_id = agent_action_group_response['agentActionGroup']['actionGroupId']
action_group_name = agent_action_group_response['agentActionGroup']['actionGroupName']

response = bedrock_agent_client.update_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupId= action_group_id,
    actionGroupName=action_group_name,
    actionGroupExecutor={
        'lambda': lambda_function['FunctionArn']
    },
    apiSchema={
        's3': {
            's3BucketName': bucket_name,
            's3ObjectKey': bucket_key
        }
    },
    actionGroupState='DISABLED',
)

action_group_deletion = bedrock_agent_client.delete_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupId= action_group_id
)

In [ ]:
agent_alias_deletion = bedrock_agent_client.delete_agent_alias(
    agentId=agent_id,
    agentAliasId=agent_alias['agentAlias']['agentAliasId']
)

In [ ]:
agent_deletion = bedrock_agent_client.delete_agent(
    agentId=agent_id
)

In [ ]:
# Delete Lambda function
lambda_client.delete_function(
    FunctionName=lambda_name
)

In [ ]:
# Empty and delete S3 Bucket

objects = s3_client.list_objects(Bucket=bucket_name)  
if 'Contents' in objects:
    for obj in objects['Contents']:
        s3_client.delete_object(Bucket=bucket_name, Key=obj['Key']) 
s3_client.delete_bucket(Bucket=bucket_name)

In [ ]:
# Delete IAM Roles and policies

for policy in [bedrock_agent_bedrock_allow_policy_name, bedrock_agent_s3_allow_policy_name]:
    iam_client.detach_role_policy(RoleName=agent_role_name, PolicyArn=f'arn:aws:iam::{account_id}:policy/{policy}')
    
iam_client.detach_role_policy(RoleName=lambda_role_name, PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole')

for role_name in [agent_role_name, lambda_role_name]:
    iam_client.delete_role(
        RoleName=role_name
    )

for policy in [agent_bedrock_policy, agent_s3_schema_policy]:
    iam_client.delete_policy(
        PolicyArn=policy['Policy']['Arn']
)

## Happy coding!